In [7]:
import sys

print(sys.executable)
#sys.path.append("/Applications/Netgen.app/Contents/MacOS")
#print(sys.path)

/usr/local/opt/python/bin/python3.7


In [ ]:
import netgen.gui
from ngsolve import *
import netgen.geom2d as geom2dim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Setup the mesh
mesh_size = 2e-1

geo = geom2dim.SplineGeometry()

geo.AddRectangle((0,0), (1,1), bcs=['b','r','t','l'], leftdomain=1)
geo.AddRectangle((0.3,0.5), (0.5,0.7), bcs=['b1','r1','t1','l1'], leftdomain=2, rightdomain=1)
geo.SetMaterial(1, "d2")
geo.SetMaterial(2, "d1")
mesh = Mesh(geo.GenerateMesh(maxh=mesh_size)) # standard mesh available
print("Number of vertices: ", mesh.nv)
print("Number of elements: ", mesh.ne)
Draw(mesh)
#Draw(geo)

In [ ]:
## Refine mesh ##
mesh.Refine()
Draw(mesh)

In [ ]:
####### Setup #######
k = 1
#fes1 = H1(mesh, definedon="d1", order=k, dirichlet='b1|r1|t1|l1')
#fes2 = H1(mesh, definedon="d2", order=k, dirichlet='b|r|t|l')
fes = H1(mesh, order=k, dirichlet='b|r|t|l')

## Set trial and test function
#u1 = fes1.TrialFunction()
#v1 = fes1.TestFunction()
#u2 = fes2.TrialFunction()
#v2 = fes2.TestFunction()
u, v = fes.TnT()

# Set right hand side
f = LinearForm(fes)
f += 1*v*dx(definedon=mesh.Materials("d1"))
f += 0*v*dx(definedon=mesh.Materials("d2"))
#rhs1 = LinearForm(fes1)
#rhs1 += 1*v1*dx
#rhs2 = LinearForm(fes2)
#rhs2 += 0*v2*dx

# The bilinear form
A = BilinearForm(fes, symmetric=True)
A += 1*grad(u)*grad(v)*dx(definedon=mesh.Materials("d1"))
A += 10*grad(u)*grad(v)*dx(definedon=mesh.Materials("d2"))
#A1 = BilinearForm(fes1, symmetric=True)
#A1 += 1*grad(u1)*grad(v1)*dx
#A2 = BilinearForm(fes2, symmetric=True)
#A2 += 10*grad(u2)*grad(v2)*dx

# Now assemble the system of equations
with TaskManager():
    f.Assemble()
    A.Assemble()
    
###### Calculations #######
# Calculate the solution field (function)
gf = GridFunction(fes)
gf.vec.data = A.mat.Inverse(fes.FreeDofs(), inverse='sparsecholesky') * f.vec


# Calculate temperature flux through left and right boundaries
#dudx = GridFunction(fes)
#dudx.Set(grad(gf)[0])

# Interpolation and integration by "hand"
#pts = [(0,i) for i in np.arange(0, 1+mesh_size, mesh_size)]
#vals = [grad(gf)(mesh(x,y))[0] for x, y in pts]
#print("vals:", vals)
# Integration using trapezoidal rule for boundary "left"
#sum = 0
#for val1, val2 in zip(vals[:-1], vals[1:]):
#    sum += (val1 + val2)/2*mesh_size

# Option 1:
# Wl = Integrate(dudx, mesh, BND, region_wise=True)
# BND... evaluate integral on boundary

# Option 2:
#Wl = Integrate(dudx, mesh, definedon=mesh.Boundaries('left'))
#Wr = Integrate(dudx, mesh, definedon=mesh.Boundaries('right'))
#Wd = Wl + Wr

In [ ]:
# Save figure as png
fig.savefig("./Ex2.4_flux-error.png")

In [ ]:
# Draw the latest solution and mesh
Draw(gf, mesh, 'grid_function')
#Draw(grad(gf), mesh, 'gradient_gf')